
Download the dataset and run RFM analysis. In each sub-segment, divide the users into 4 classes (as in the lesson). Count the number of days since the last purchase from the maximum purchase date in the dataset.

1. What is the maximum number of purchases made by one user?
2. What is the upper limit for the amount of purchases from users with class 4 in subsegment M? (In other words: users whose purchase amount is from 0 to X fall into class 4 in the M subsegment)
3. What is the lower bound on the number of purchases from users with class 1 in subsegment F?
4. What is the upper bound on the number of purchases from users with class 2 in subsegment R?
5. How many users fell into segment 111?
6. How many users fell into segment 311?
7. Which RFM segment has the largest number of users?
8. Which RFM segment has the smallest number of users?
9. How many users fell into the smallest segment?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import timedelta
import datetime as dt
%matplotlib inline 
import calendar

In [2]:
df = pd.read_csv('./RFM_ht_data.csv', parse_dates=['InvoiceDate'], dtype={"InvoiceNo": str, "CustomerCode": str, "Amount": "float64"})

In [3]:
df.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [4]:
df.dtypes

InvoiceNo               object
CustomerCode            object
InvoiceDate     datetime64[ns]
Amount                 float64
dtype: object

In [5]:
last_date = df.InvoiceDate.max()
last_date

Timestamp('2020-09-30 00:00:00')

In [6]:
df.columns

Index(['InvoiceNo', 'CustomerCode', 'InvoiceDate', 'Amount'], dtype='object')

In [7]:
rename_columns = {}
for col in df.columns:
    rename_columns[col] = col.lower()

In [8]:
df = df.rename(columns=rename_columns)
df.head(2)

,invoiceno,customercode,invoicedate,amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74


In [9]:
df = df.rename(columns={
    'invoiceno':'invoice_no',
    'customercode' : 'customer_code',
    'invoicedate' : 'invoice_date',
    'amount':'amount'})

df.head()

,invoice_no,customer_code,invoice_date,amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [10]:
rfm_df = df.groupby(by='customer_code', as_index=False) \
    .agg(
        {
            'invoice_date': lambda x: (last_date - x.max()).days, # Recency 
            'invoice_no': len, # Frequency
            'amount': 'sum' # Monetary
        } 
    )

In [11]:
rfm_df

,customer_code,invoice_date,invoice_no,amount
0,02213019,19,1,1609.20
1,02213042,22,3,9685.48
2,02213071,29,1,415.00
3,02213088,23,1,305.00
4,02213092,25,1,1412.88
...,...,...,...,...
123728,99099927,10,1,961.10
123729,99099936,0,1,1521.78
123730,99099959,8,2,1444.56
123731,99099963,19,1,3018.91


In [12]:
rfm_df = rfm_df.rename(columns={
    'invoice_date':'recency',
    'invoice_no' : 'frequency',
    'amount':'monetary'})

rfm_df.head()

,customer_code,recency,frequency,monetary
0,02213019,19,1,1609.20
1,02213042,22,3,9685.48
2,02213071,29,1,415.00
3,02213088,23,1,305.00
4,02213092,25,1,1412.88


In [13]:
df_quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
df_quantiles

C:\Users\Ann Zalivatskaya\AppData\Local\Temp\ipykernel_17944\423560410.py:1: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])


,recency,frequency,monetary
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [14]:
def R(value, parameter_name, quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4


def FM(value, parameter_name, quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1

In [15]:
rfm_df['r'] = rfm_df.recency.apply(R, args=('recency', df_quantiles))
rfm_df['f'] = rfm_df.frequency.apply(FM, args=('frequency', df_quantiles))
rfm_df['m'] = rfm_df.monetary.apply(FM, args=('monetary', df_quantiles))

In [16]:
rfm_df

,customer_code,recency,frequency,monetary,r,f,m
0,02213019,19,1,1609.20,4,4,3
1,02213042,22,3,9685.48,4,2,1
2,02213071,29,1,415.00,4,4,4
3,02213088,23,1,305.00,4,4,4
4,02213092,25,1,1412.88,4,4,3
...,...,...,...,...,...,...,...
123728,99099927,10,1,961.10,3,4,3
123729,99099936,0,1,1521.78,1,4,3
123730,99099959,8,2,1444.56,2,3,3
123731,99099963,19,1,3018.91,4,4,2


In [17]:
rfm_df['RFM_Score'] = rfm_df.r.map(str) + rfm_df.f.map(str) + rfm_df.m.map(str)

rfm_df.head()

,customer_code,recency,frequency,monetary,r,f,m,RFM_Score
0,02213019,19,1,1609.20,4,4,3,443
1,02213042,22,3,9685.48,4,2,1,421
2,02213071,29,1,415.00,4,4,4,444
3,02213088,23,1,305.00,4,4,4,444
4,02213092,25,1,1412.88,4,4,3,443


# Task 1
What is the maximum number of purchases made by one user?

In [18]:
df_invoices = df.groupby(by='customer_code', as_index=False) \
    .agg({'invoice_no':'count'}) \
    .sort_values(by='invoice_no', ascending=False) \
    .rename(columns={'invoice_no':'invoice_counts'})

df_invoices.head()

,customer_code,invoice_counts
89388,19057820,204
44594,13215452,113
10347,13032521,106
97077,19080880,99
119951,99003061,90


Answer:

In [19]:
df_invoices[df_invoices.invoice_counts == df_invoices.invoice_counts.max()]

,customer_code,invoice_counts
89388,19057820,204


# Task 2
What is the upper limit for the amount of purchases from users with class 4 in subsegment M? (In other words: users whose purchase amount is from 0 to X fall into class 4 in the M subsegment)

In [20]:
rfm_df.monetary.quantile(q=[0.25,0.5,0.75])

0.25     765.00
0.50    1834.48
0.75    4008.84
Name: monetary, dtype: float64

Answer: 765

# Task 3
What is the lower bound on the number of purchases from users with class 1 in subsegment F?

In [21]:
rfm_df.frequency.quantile(q=[0.25,0.5,0.75])

0.25    1.0
0.50    2.0
0.75    3.0
Name: frequency, dtype: float64

In [22]:
rfm_df.query('frequency > 3').min()

customer_code    02213129
recency                 0
frequency               4
monetary            265.0
r                       1
f                       1
m                       1
RFM_Score             111
dtype: object

Answer: 4

# Task 4
What is the maximum number of days that can elapse since the last purchase for a user to fall into class 2 in subsegment R?

In [23]:
rfm_df.recency.quantile(q=[0.25,0.5,0.75])

0.25     2.0
0.50     8.0
0.75    16.0
Name: recency, dtype: float64

Answer: 8

# Answer 5
How many users fell into segment 111?

In [24]:
rfm_df

,customer_code,recency,frequency,monetary,r,f,m,RFM_Score
0,02213019,19,1,1609.20,4,4,3,443
1,02213042,22,3,9685.48,4,2,1,421
2,02213071,29,1,415.00,4,4,4,444
3,02213088,23,1,305.00,4,4,4,444
4,02213092,25,1,1412.88,4,4,3,443
...,...,...,...,...,...,...,...,...
123728,99099927,10,1,961.10,3,4,3,343
123729,99099936,0,1,1521.78,1,4,3,143
123730,99099959,8,2,1444.56,2,3,3,233
123731,99099963,19,1,3018.91,4,4,2,442


In [25]:
rfm_df_count = rfm_df \
    .groupby(by='RFM_Score') \
    .agg({'customer_code':'count'}) \
    .rename(columns={'customer_code':'customers_counts'}) \
    .sort_values(by='customers_counts', ascending=False)

rfm_df_count

,customers_counts
RFM_Score,
444,10624
111,9705
443,6729
344,6593
211,5847
...,...
424,63
214,60
114,60


In [26]:
rfm_df_count.loc['111']

customers_counts    9705
Name: 111, dtype: int64

# Task 6
How many users fell into segment 311?

In [27]:
rfm_df_count.loc['311']

customers_counts    1609
Name: 311, dtype: int64

# Task 7
Which RFM segment has the largest number of users?

In [28]:
rfm_df_count.customers_counts.max()

10624

In [29]:
rfm_df_count[rfm_df_count.customers_counts == rfm_df_count.customers_counts.max()]

,customers_counts
RFM_Score,
444,10624


# Task 8
Which RFM segment has the smallest number of users?


In [30]:
rfm_df_count[rfm_df_count.customers_counts == rfm_df_count.customers_counts.min()]

,customers_counts
RFM_Score,
414,2
